In [2]:
!pip install plotly
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 33.8 MB/s eta 0:00:00


In [3]:
import plotly
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-3-e952e1cb51bd>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-3-e952e1cb51bd>:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [ ]:
games = pd.read_csv("games.csv")
players = pd.read_csv("players.csv")
plays = pd.read_csv("plays.csv")
tackles = pd.read_csv("tackles.csv")

conf_div = pd.read_csv("conf_div.csv")
players_within_3_5_wks_7_9 = pd.read_pickle("players_within_3_5_wks_7_9.pkl")

all_players_sorted_7 = pd.read_pickle("all_players_sorted_7.pkl")
all_players_sorted_8 = pd.read_pickle("all_players_sorted_8.pkl")
all_players_sorted_9 = pd.read_pickle("all_players_sorted_9.pkl")

results_df_7_9 = pd.read_csv("results_df_7_9.csv", index_col=0)
tackle_data = pd.read_csv("tackle_data.csv")

In [89]:
players_within_3_5_wks_7_9

,gameId,playId,frameId,nflId,displayName,new_pos,within_3_yds,within_5_yds,defensiveTeam,possessionTeam
0,2022102000,80,18,37084.0,J.J. Watt,DL,0,0,ARI,NO
1,2022102000,80,18,37110.0,Andy Dalton,QB,1,1,ARI,NO
2,2022102000,80,18,41532.0,James Hurst,OL,0,0,ARI,NO
3,2022102000,80,18,44844.0,Ryan Ramczyk,OL,0,0,ARI,NO
4,2022102000,80,18,44848.0,Budda Baker,DB,0,0,ARI,NO
...,...,...,...,...,...,...,...,...,...,...
1360277,2022110700,3787,40,54541.0,Travis Jones,DL,1,1,BAL,NO
1360278,2022110700,3787,40,53460.0,Odafe Oweh,LB,0,0,BAL,NO
1360279,2022110700,3787,40,52666.0,Calvin Throckmorton,OL,1,1,BAL,NO
1360280,2022110700,3787,40,54606.0,Damarion Williams,DB,0,0,BAL,NO


In [9]:
ARI_players = ['Antwaun Woods', 'J.J. Watt', 'Jonathan Ledbetter', 'Leki Fotu', 'Michael Dogbe', 'Rashard Lawrence', 'Zach Allen', 'Ben Niemann', 'Cameron Thomas', 'Ezekiel Turner', 'Jesse Luketa',
               'Markus Golden', 'Myjai Sanders', 'Tanner Vallejo', 'Victor Dimukeje', 'Zaven Collins', 'Antonio Hamilton', 'Budda Baker', 'Byron Murphy', 'Chris Banjo', 'Isaiah Simmons', 'Jalen Thompson', 'Marco Wilson']

In [10]:
ARI_players_within_3_5_wks_7_9 = players_within_3_5_wks_7_9[players_within_3_5_wks_7_9['defensiveTeam'] == 'ARI']

In [11]:
name_pos_plays = ARI_players_within_3_5_wks_7_9.groupby(['displayName', 'new_pos'])['playId'].nunique().reset_index().sort_values(['playId'], ascending=False).reset_index(drop=True)
name_pos_plays = name_pos_plays.rename(columns={'displayName': 'Name', 'new_pos': 'Pos', 'playId': '# Plays'})

In [12]:
name_pos_sec = ARI_players_within_3_5_wks_7_9.groupby(['displayName', 'new_pos'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
name_pos_sec = name_pos_sec.rename(columns={'displayName': 'Name', 'new_pos': 'Pos', 'frameId': 'Total Time'})
name_pos_sec['Total Time'] = name_pos_sec['Total Time'] / 10

In [13]:
name_pos_wi_5 = ARI_players_within_3_5_wks_7_9[ARI_players_within_3_5_wks_7_9['within_5_yds'] == 1].groupby(['displayName', 'new_pos'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
name_pos_wi_5 = name_pos_wi_5.rename(columns={'displayName': 'Name', 'new_pos': 'Pos', 'frameId': '<5yds'})
name_pos_wi_5['<5yds'] = name_pos_wi_5['<5yds'] / 10

In [14]:
name_pos_wi_3 = ARI_players_within_3_5_wks_7_9[ARI_players_within_3_5_wks_7_9['within_3_yds'] == 1].groupby(['displayName', 'new_pos'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
name_pos_wi_3 = name_pos_wi_3.rename(columns={'displayName': 'Name', 'new_pos': 'Pos', 'frameId': '<3yds'})
name_pos_wi_3['<3yds'] = name_pos_wi_3['<3yds'] / 10

In [15]:
all_players_sorted_7_9 = pd.concat([all_players_sorted_7, all_players_sorted_8, all_players_sorted_9])
ARI_aps_7_9 = all_players_sorted_7_9[all_players_sorted_7_9['displayName'].isin(ARI_players)]

In [16]:
name_pos_nearest = ARI_aps_7_9[ARI_aps_7_9['rank'] == 2].groupby(['displayName'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
name_pos_nearest = name_pos_nearest.rename(columns={'displayName': 'Name', 'frameId': 'Closest Def'})
name_pos_nearest['Closest Def'] = name_pos_nearest['Closest Def'] / 10

In [17]:
name_pos_2nd_nearest = ARI_aps_7_9[ARI_aps_7_9['rank'] == 3].groupby(['displayName'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
name_pos_2nd_nearest = name_pos_2nd_nearest.rename(columns={'displayName': 'Name', 'frameId': '2nd Def'})
name_pos_2nd_nearest['2nd Def'] = name_pos_2nd_nearest['2nd Def'] / 10

In [18]:
name_pos_3rd_nearest = ARI_aps_7_9[ARI_aps_7_9['rank'] == 4].groupby(['displayName'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
name_pos_3rd_nearest = name_pos_3rd_nearest.rename(columns={'displayName': 'Name', 'frameId': '3rd Def'})
name_pos_3rd_nearest['3rd Def'] = name_pos_3rd_nearest['3rd Def'] / 10

In [19]:
ARI_DTC = tackle_data[tackle_data['defensiveTeam'] == 'ARI']

name_pos_sum_DTC = ARI_DTC.groupby(['def1_name'])['new_prop'].sum().reset_index().sort_values(['new_prop'], ascending=False).reset_index(drop=True)
name_pos_sum_DTC = name_pos_sum_DTC.rename(columns={'def1_name': 'Name', 'new_prop': 'Sum DTC'})
name_pos_sum_DTC['Sum DTC'] = round(name_pos_sum_DTC['Sum DTC'], 2)

name_pos_num_DTC = ARI_DTC.groupby(['def1_name'])['new_prop'].count().reset_index().sort_values(['new_prop'], ascending=False).reset_index(drop=True)
name_pos_num_DTC = name_pos_num_DTC.rename(columns={'def1_name': 'Name', 'new_prop': 'Num DTC'})

name_pos_num_DTC

,Name,Num DTC
0,Budda Baker,15
1,Markus Golden,12
2,Zaven Collins,11
3,Zach Allen,11
4,Jalen Thompson,10
5,Isaiah Simmons,9
6,J.J. Watt,8
7,Tanner Vallejo,7
8,Victor Dimukeje,6
9,Leki Fotu,6


In [20]:
team_breakdown = name_pos_plays.merge(name_pos_sec).merge(name_pos_wi_5).merge(name_pos_wi_3).merge(name_pos_nearest).merge(name_pos_2nd_nearest).merge(name_pos_3rd_nearest).merge(name_pos_num_DTC).merge(name_pos_sum_DTC)

In [21]:
team_breakdown['DTC/<5yds'] = round(team_breakdown['Sum DTC'] / team_breakdown['<5yds'], 2)
team_breakdown['DTC/<3yds'] = round(team_breakdown['Sum DTC'] / team_breakdown['<3yds'], 2)

In [22]:
team_breakdown

,Name,Pos,# Plays,Total Time,<5yds,<3yds,Closest Def,2nd Def,3rd Def,Num DTC,Sum DTC,DTC/<5yds,DTC/<3yds
0,Zaven Collins,LB,66,220.9,106.0,52.1,17.3,23.9,20.2,11,4.65,0.04,0.09
1,Byron Murphy,DB,66,220.9,15.3,7.8,0.5,1.6,3.3,1,0.38,0.02,0.05
2,Budda Baker,DB,65,217.2,87.5,57.5,20.3,13.0,15.5,15,8.93,0.10,0.16
3,Jalen Thompson,DB,65,217.2,56.9,41.9,15.2,13.5,8.2,10,3.93,0.07,0.09
4,Zach Allen,DL,51,170.3,99.6,52.4,25.6,21.6,22.3,11,4.40,0.04,0.08
5,Marco Wilson,DB,51,172.7,31.9,16.0,8.4,5.6,5.6,4,2.81,0.09,0.18
6,J.J. Watt,DL,48,162.8,93.1,42.3,22.3,19.8,23.5,8,3.71,0.04,0.09
7,Victor Dimukeje,LB,42,137.6,54.0,23.1,15.8,13.2,15.5,6,3.85,0.07,0.17
8,Markus Golden,LB,42,146.3,77.9,40.0,18.6,16.0,10.7,12,5.52,0.07,0.14
9,Leki Fotu,DL,42,142.5,72.9,38.9,14.3,17.7,20.0,6,2.68,0.04,0.07


In [23]:
# Create the bubble chart
fig = px.scatter(
    team_breakdown,
    x='<3yds',
    y='Total Time',
    size='DTC/<3yds',  # Size of bubbles
    color='Pos',  # Color of bubbles (optional, can be another variable)
    hover_name='Name',  # Hover text
    size_max=60  # Maximum bubble size
)

# Update layout if necessary
fig.update_layout(
    title='Team Breakdown Bubble Chart',
    xaxis_title='<3 Yards Time',
    yaxis_title='Total Time',
    legend_title='Position'
)

fig.show()

In [25]:
d_players_within_3_5_wks_7_9 = players_within_3_5_wks_7_9[players_within_3_5_wks_7_9['new_pos'].isin(['DB', 'DL', 'LB'])]

In [26]:
b = d_players_within_3_5_wks_7_9.groupby(['displayName', 'new_pos'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
b = b.rename(columns={'frameId': 'sec'})
b['sec'] = b['sec'] / 10

In [27]:
a = d_players_within_3_5_wks_7_9[d_players_within_3_5_wks_7_9['within_5_yds'] == 1].groupby(['displayName', 'new_pos'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
a = a.rename(columns={'frameId': '<5yds'})
a['<5yds'] = a['<5yds'] / 10

c = d_players_within_3_5_wks_7_9[d_players_within_3_5_wks_7_9['within_3_yds'] == 1].groupby(['displayName', 'new_pos'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
c = c.rename(columns={'frameId': '<3yds'})
c['<3yds'] = c['<3yds'] / 10

In [28]:
d = tackle_data.groupby(['def1_name'])['new_prop'].sum().reset_index().sort_values(['new_prop'], ascending=False).reset_index(drop=True)
d = d.rename(columns={'def1_name': 'displayName'})

In [29]:
dd = tackle_data.groupby(['def1_name'])['neg epa'].sum().reset_index().sort_values(['neg epa'], ascending=False).reset_index(drop=True)
dd = dd.rename(columns={'def1_name': 'displayName', 'neg epa': '# DTC (-EPA)'})

In [30]:
e = a.merge(b).merge(c).merge(d).merge(dd)

e['DTC/<5yds'] = round(e['new_prop'] / e['<5yds'], 3)
e['DTC/<3yds'] = round(e['new_prop'] / e['<3yds'], 3)
e['# DTC (-EPA)'] = round(e['# DTC (-EPA)'], 3)

e

,displayName,new_pos,<5yds,sec,<3yds,new_prop,# DTC (-EPA),DTC/<5yds,DTC/<3yds
0,Kenny Clark,DL,159.0,233.8,90.4,6.531975,9,0.041,0.072
1,Grover Stewart,DL,156.1,192.0,100.7,10.078365,14,0.065,0.100
2,Mykal Walker,LB,156.0,230.4,100.7,11.597233,14,0.074,0.115
3,Yetur Gross-Matos,LB,154.0,212.9,102.3,8.182828,9,0.053,0.080
4,Shaq Thompson,LB,146.9,245.4,106.0,13.111862,8,0.089,0.124
...,...,...,...,...,...,...,...,...,...
603,Tarvarius Moore,DB,2.2,6.7,1.3,0.399985,0,0.182,0.308
604,DeAndre Houston-Carson,DB,2.0,5.7,1.6,0.232625,0,0.116,0.145
605,Lonnie Johnson,DB,1.9,4.4,1.7,1.000000,0,0.526,0.588
606,Ahkello Witherspoon,DB,1.8,18.9,0.7,0.636998,0,0.354,0.910


In [31]:
e_100 = e.sort_values(['<3yds'], ascending=False).head(100)

In [32]:
e_100

,displayName,new_pos,<5yds,sec,<3yds,new_prop,# DTC (-EPA),DTC/<5yds,DTC/<3yds
4,Shaq Thompson,LB,146.9,245.4,106.0,13.111862,8,0.089,0.124
3,Yetur Gross-Matos,LB,154.0,212.9,102.3,8.182828,9,0.053,0.080
2,Mykal Walker,LB,156.0,230.4,100.7,11.597233,14,0.074,0.115
1,Grover Stewart,DL,156.1,192.0,100.7,10.078365,14,0.065,0.100
0,Kenny Clark,DL,159.0,233.8,90.4,6.531975,9,0.041,0.072
...,...,...,...,...,...,...,...,...,...
111,Roquan Smith,LB,75.1,142.2,45.4,8.295737,10,0.110,0.183
163,John Ridgeway,DL,61.4,71.8,45.1,2.733643,6,0.045,0.061
102,Jordyn Brooks,LB,78.3,149.1,44.8,7.910805,12,0.101,0.177
115,Abdullah Anderson,DL,73.7,97.1,44.5,3.173765,6,0.043,0.071


In [33]:
# Create the bubble chart
fig = px.scatter(
    e_100,
    x='<3yds',
    y='sec',
    size='DTC/<3yds',  # Size of bubbles
    color='new_pos',  # Color of bubbles (optional, can be another variable)
    hover_name='displayName',  # Hover text
    size_max=30  # Maximum bubble size
)

# Update layout
fig.update_layout(
    title='Team Breakdown Bubble Chart',
    xaxis_title='<3 Yards Time',
    yaxis_title='Total Time',
    legend_title='Position'
)

fig.show()

In [ ]:
team_b = d_players_within_3_5_wks_7_9.groupby(['defensiveTeam', 'new_pos'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
team_b = team_b.rename(columns={'frameId': 'sec'})
team_b['sec'] = team_b['sec'] / 10

team_a = d_players_within_3_5_wks_7_9[d_players_within_3_5_wks_7_9['within_5_yds'] == 1].groupby(['defensiveTeam', 'new_pos'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
team_a = team_a.rename(columns={'frameId': '<5yds'})
team_a['<5yds'] = team_a['<5yds'] / 10

team_c = d_players_within_3_5_wks_7_9[d_players_within_3_5_wks_7_9['within_3_yds'] == 1].groupby(['defensiveTeam', 'new_pos'])['frameId'].count().reset_index().sort_values(['frameId'], ascending=False).reset_index(drop=True)
team_c = team_c.rename(columns={'frameId': '<3yds'})
team_c['<3yds'] = team_c['<3yds'] / 10

team_d = tackle_data.groupby(['defensiveTeam', 'def1_pos'])['new_prop'].sum().reset_index().sort_values(['new_prop'], ascending=False).reset_index(drop=True)
team_d = team_d.rename(columns= {'def1_pos': 'new_pos'})

team_dd = tackle_data.groupby(['defensiveTeam', 'def1_pos'])['neg epa'].sum().reset_index().sort_values(['neg epa'], ascending=False).reset_index(drop=True)
team_dd = team_dd.rename(columns={'neg epa': '# DTC (-EPA)', 'def1_pos': 'new_pos'})

team_e = team_a.merge(team_b).merge(team_c).merge(team_d).merge(team_dd)

team_e['DTC/<5yds'] = round(team_e['new_prop'] / team_e['<5yds'], 3)
team_e['DTC/<3yds'] = round(team_e['new_prop'] / team_e['<3yds'], 3)
team_e['# DTC (-EPA)'] = round(team_e['# DTC (-EPA)'], 3)

num_plays = team_e.groupby(['defensiveTeam'])['new_prop'].sum().reset_index()
num_plays = num_plays.rename(columns={'new_prop': 'num_plays'})

team_e = team_e.merge(num_plays)
team_e['Pos % of DTC'] = round(team_e['new_prop'] / team_e['num_plays'], 3)

conf_div = conf_div.rename(columns={'Team': 'defensiveTeam'})
team_e = team_e.merge(conf_div)
team_e['Team-Pos'] = team_e['defensiveTeam'] + "-" + team_e['new_pos']

team_e

In [ ]:
# App that's interactive, but not compatible with Kagggle

app = dash.Dash(__name__)

# App Layout
app.layout = html.Div([
    dcc.Dropdown(
        id='conference-filter',
        options=[{'label': 'All', 'value': 'All'}] +
                [{'label': conf, 'value': conf} for conf in team_e['Conference'].unique()],
        value='All',
        placeholder="Select a conference"
    ),
    dcc.Dropdown(
        id='division-filter',
        options=[{'label': 'All', 'value': 'All'}] +
                [{'label': div, 'value': div} for div in team_e['Division'].unique()],
        value='All',
        placeholder="Select a division"
    ),
    dcc.Dropdown(
        id='position-filter',
        options=[{'label': 'All', 'value': 'All'}] +
                [{'label': pos, 'value': pos} for pos in team_e['new_pos'].unique()],
        value='All',
        placeholder="Select a position"
    ),
    dcc.Graph(id='bubble-chart')
])


# Callback for updating the bubble chart
@app.callback(
    Output('bubble-chart', 'figure'),
    [Input('conference-filter', 'value'),
     Input('division-filter', 'value'),
     Input('position-filter', 'value')]
)
def update_chart(selected_conference, selected_division, selected_position):
    filtered_data = team_e.copy()

    # Apply filters only if the option is not 'All'
    if selected_conference != 'All':
        filtered_data = filtered_data[filtered_data['Conference'] == selected_conference]
    if selected_division != 'All':
        filtered_data = filtered_data[filtered_data['Division'] == selected_division]
    if selected_position != 'All':
        filtered_data = filtered_data[filtered_data['new_pos'] == selected_position]




    # Create the bubble chart
    fig = px.scatter(
        filtered_data,
        x='<3yds',
        y='sec',
        size='Pos % of DTC',
        color='new_pos',
        hover_name='Team-Pos',
        text='defensiveTeam',
        size_max=30
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
all_predicted_7 = results_df_7_9[['gameId', 'playId', 'frameId', 'Predicted']].merge(all_players_sorted_7, on=['gameId', 'playId', 'frameId'], how='inner')



def polar_to_cartesian(row):
    # Convert angle to radians and calculate x, y coordinates
    angle_rad = np.radians(row['angle_for_min_distance'])
    x = row['std_x_nb'] + row['min_distance'] * np.cos(angle_rad)
    y = row['std_y_nb'] + row['min_distance'] * np.sin(angle_rad)
    return x, y

def polar_to_cartesian_2(row):
    # Convert angle to radians and calculate x, y coordinates
    angle_rad = np.radians(row['angle_for_second_min_distance'])
    x = row['std_x_nb'] + row['second_min_distance'] * np.cos(angle_rad)
    y = row['std_y_nb'] + row['second_min_distance'] * np.sin(angle_rad)
    return x, y


df_7 = all_predicted_7[['gameId','playId','frameId','Predicted',
 'dist_to_ball',
 'ang_to_ball',
 'within_3_yds',
 'within_5_yds',
 'frame_since_bc',
 'rank',
 'new_pos',
 'x_no_1',
 'y_no_1',
 'min_distance',
 'angle_for_min_distance',
 'second_min_distance',
 'angle_for_second_min_distance',
 'max_distance',
 'actual_dist_from_final',
 'std_x_nb',
 'std_y_nb',
 'std_bc_x',
 'std_bc_y']]


df_7['closest_o_x'], df_7['closest_o_y'] = zip(*all_predicted_7.apply(polar_to_cartesian, axis=1))
df_7['second_closest_o_x'], df_7['second_closest_o_y'] = zip(*all_predicted_7.apply(polar_to_cartesian_2, axis=1))

df_7 = df_7[df_7['rank'] > 1.5]

In [90]:
import plotly.express as px

# Sample Data Filtering - Modify as needed
selected_conference = 'YourConference'  # Replace with your desired conference
selected_division = 'YourDivision'  # Replace with your desired division
selected_position = 'YourPosition'  # Replace with your desired position

fig = px.scatter(
    team_e,
    x='<3yds',
    y='sec',
    size='Pos % of DTC',
    color='new_pos',
    hover_name='Team-Pos',
    text='defensiveTeam',
    size_max=30
)


fig.update_layout(
    title='DTC per second within 3 yards of ball carrier',
    xaxis_title='Seconds Within 3 Yards of Ball Carrier',
    yaxis_title='Total Time in seconds',
    legend_title='Position'
)

fig.show()

In [88]:
import plotly.graph_objects as go

# Filter for a specific play
specific_game_id = 2022102300
specific_play_id = 388
play_data = df_7[(df_7['gameId'] == specific_game_id) & (df_7['playId'] == specific_play_id)]

# Get unique frames
unique_frames = sorted(play_data['frameId'].unique())

# Initialize the figure
fig = go.Figure()

# Create frames for each unique frame in the play
frames = []
for frame_id in unique_frames:
    frame_data = play_data[play_data['frameId'] == frame_id]
    frame_traces = []

    # Add trace for defensive player
    frame_traces.append(
        go.Scatter(
            x=frame_data['std_x_nb'],
            y=frame_data['std_y_nb'],
            mode='markers',
            marker=dict(size=10, color='blue', symbol='circle'),
            name='Defensive Player'
        )
    )

    # Add trace for ball carrier
    frame_traces.append(
        go.Scatter(
            x=frame_data['std_bc_x'],
            y=frame_data['std_bc_y'],
            mode='markers',
            marker=dict(size=12, color='red', symbol='star'),
            name='Ball Carrier'
        )
    )

    # Add traces for offensive players
    for col_x, col_y in [('x_no_1', 'y_no_1'), ('closest_o_x', 'closest_o_y'), ('second_closest_o_x', 'second_closest_o_y')]:
        if col_x in frame_data and col_y in frame_data:
            frame_traces.append(
                go.Scatter(
                    x=frame_data[col_x],
                    y=frame_data[col_y],
                    mode='markers',
                    marker=dict(size=8, color='green', symbol='square'),
                    name='Offensive Player'
                )
            )


    frames.append(go.Frame(data=frame_traces, name=str(frame_id)))

    # Calculate positions for vertical lines
    ball_carrier_x = frame_data['std_bc_x'].iloc[0]  # Assuming there's only one ball carrier per frame
    actual_line_x = ball_carrier_x - frame_data['actual_dist_from_final'].iloc[0]
    predicted_line_x = ball_carrier_x - frame_data['Predicted'].iloc[0]



    # Add vertical lines as shapes
    frame_shapes = [
        go.layout.Shape(
            type="line",
            x0=actual_line_x, y0=0, x1=actual_line_x, y1=55,  # y1 should be the top of your y-axis
            line=dict(color="yellow", width=2)
        ),
        go.layout.Shape(
            type="line",
            x0=predicted_line_x, y0=0, x1=predicted_line_x, y1=55,
            line=dict(color="white", width=2)
        )
    ]



    # Add annotations for the actual and predicted lines
    frame_annotations = [
        go.layout.Annotation(
            x=actual_line_x,
            y=max(frame_data['std_y_nb']),  # or another suitable y-coordinate
            text="Actual",
            showarrow=True,
            arrowhead=1,
            ax=-150  # Adjust this for proper positioning
        ),
        go.layout.Annotation(
            x=predicted_line_x,
            y=max(frame_data['std_y_nb']),  # or another suitable y-coordinate
            text="Predicted",
            showarrow=True,
            arrowhead=1,
            ax=-30  # Adjust this for proper positioning
        )
    ]

    # Add this set of traces as a frame with shapes and annotations
    frames.append(go.Frame(data=frame_traces, name=str(frame_id), layout=go.Layout(shapes=frame_shapes, annotations=frame_annotations)))


# Set frames to the figure
fig.frames = frames

# Update layout for animation
fig.update_layout(
    title=f'G.Edwards for 12 yards, BAL vs. CLE Q1 7:10',
    xaxis=dict(title='Yards From Endzone', range=[0, 110]),
    yaxis=dict(title='Yards Sideline to Sideline', range=[0, 55]),
    updatemenus=[{
        'type': 'buttons',
        'buttons': [{
            'label': 'Play',
            'method': 'animate',
            'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}]
        }]
    }],
    sliders=[{
        'steps': [{'method': 'animate', 'args': [[str(frame_id)], {'mode': 'immediate', 'frame': {'duration': 500, 'redraw': True}}], 'label': str(frame_id)} for frame_id in unique_frames]
    }]
)

# Initialize the figure with the first frame's data
initial_frame_data = play_data[play_data['frameId'] == unique_frames[0]]
fig.add_traces(frames[0]['data'])

fig.show()